In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [4]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [0]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [6]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.csv to Dataset.csv


In [7]:
intent, unique_intent, sentences = load_dataset("Dataset.csv")

                Sentence          Intent
0       Need help pleese  commonQ.assist
1              Need help  commonQ.assist
2       I need some info  commonQ.assist
3      Will you help me?  commonQ.assist
4  What else can you do?  commonQ.assist


In [8]:
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']


In [9]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#define stemmer
stemmer = LancasterStemmer()

In [0]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [12]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


1113
[['need', 'help', 'pleese'], ['need', 'help']]


In [0]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [0]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [15]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 492 and Maximum length = 28


In [0]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [0]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [0]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [0]:
padded_doc = padding_doc(encoded_doc, max_length)

In [20]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [21]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1113, 28)


In [0]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [23]:
output_tokenizer.word_index

{'commonq.assist': 6,
 'commonq.bot': 20,
 'commonq.how': 19,
 'commonq.just_details': 21,
 'commonq.name': 18,
 'commonq.not_giving': 14,
 'commonq.query': 15,
 'commonq.wait': 2,
 'contact.contact': 10,
 'faq.aadhaar_missing': 3,
 'faq.address_proof': 11,
 'faq.application_process': 9,
 'faq.apply_register': 12,
 'faq.approval_time': 1,
 'faq.bad_service': 8,
 'faq.banking_option_missing': 4,
 'faq.biz_category_missing': 17,
 'faq.biz_new': 7,
 'faq.biz_simpler': 13,
 'faq.borrow_limit': 5,
 'faq.borrow_use': 16}

In [0]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [0]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [26]:
encoded_output.shape

(1113, 1)

In [0]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [0]:
output_one_hot = one_hot(encoded_output)

In [29]:
output_one_hot.shape

(1113, 21)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [32]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (890, 28) and train_Y = (890, 21)
Shape of val_X = (223, 28) and val_Y = (223, 21)


In [0]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [34]:
model = create_model(vocab_size, max_length)


model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           62976     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 335,061
Trainable params: 272,085
Non-trainable params: 62,976
_________________________________________________________________


In [35]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 4s 4ms/step - loss: 2.9791 - acc: 0.0944 - val_loss: 2.8677 - val_acc: 0.1749

Epoch 00001: val_loss improved from inf to 2.86768, saving model to model.h5
Epoch 2/100
890/890 [==============================] - 3s 3ms/step - loss: 2.8781 - acc: 0.1303 - val_loss: 2.7820 - val_acc: 0.1749

Epoch 00002: val_loss improved from 2.86768 to 2.78198, saving model to model.h5
Epoch 3/100
890/890 [==============================] - 3s 3ms/step - loss: 2.8122 - acc: 0.1393 - val_loss: 2.6642 - val_acc: 0.2152

Epoch 00003: val_loss improved from 2.78198 to 2.66419, saving model to model.h5
Epoch 4/100
890/890 [==============================] - 3s 3ms/step - loss: 2.7181 - acc: 0.1494 - val_loss: 2.5530 - val_acc: 0.1973

Epoch 00004: val_loss improved from 2.66419 to 2.55301, saving model to model.h5
Epoch 5/100
890/890 [==============================] - 3s 3ms/step - loss: 2.6417 - acc: 0.1742 - 

In [0]:
 model = load_model("model.h5")

In [0]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [0]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [39]:
text = "Can you help me?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['can', 'you', 'help', 'me']
commonQ.assist has confidence = 0.26955733
commonQ.name has confidence = 0.15020579
commonQ.bot has confidence = 0.132447
faq.bad_service has confidence = 0.109360725
commonQ.not_giving has confidence = 0.07746187
commonQ.wait has confidence = 0.07732106
contact.contact has confidence = 0.041986175
commonQ.query has confidence = 0.03340741
faq.biz_new has confidence = 0.026701674
faq.apply_register has confidence = 0.01643285
commonQ.just_details has confidence = 0.016130144
commonQ.how has confidence = 0.011909867
faq.banking_option_missing has confidence = 0.011591457
faq.biz_simpler has confidence = 0.007584581
faq.aadhaar_missing has confidence = 0.005653782
faq.address_proof has confidence = 0.0040731374
faq.biz_category_missing has confidence = 0.0028133863
faq.borrow_use has confidence = 0.0024139883
faq.borrow_limit has confidence = 0.002107147
faq.approval_time has confidence = 0.0004718498
faq.application_process has confidence = 0.00036874504
